In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModel, AutoTokenizer
model_name = "bert-base-uncased"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from transformers import BertTokenizerFast, EncoderDecoderModel
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizerFast.from_pretrained('mrm8488/bert-small2bert-small-finetuned-cnn_daily_mail-summarization')
model = EncoderDecoderModel.from_pretrained('mrm8488/bert-small2bert-small-finetuned-cnn_daily_mail-summarization').to(device)

def generate_summary(text):

    inputs = tokenizer([text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    output = model.generate(input_ids, attention_mask=attention_mask)

    return tokenizer.decode(output[0], skip_special_tokens=True)

text = """Thanks for joining XX Day – by far our biggest event ever! The conference was a great success and we’re thrilled you were able to participate. With all the rich technical content across 5 tracks and 80 sessions, be sure to catch the replays of sessions you may have missed.

For those seeking the Data & AI Skills Certificate or the XX Skills Certificate, please ensure you complete all the required sessions and take the knowledge quiz to earn your certificate. Remember the deadline is October 25th! For detailed information, go to:

We have one final request – your feedback is a gift. Please complete this short survey on XX Day to provide your input about the conference. It will only take a few minutes and you will be entered to win a $50 gift card. Help us make next year’s XX Day even bigger and better!
"""
generate_summary(text)


"the conference was a great success and we're thrilled you were able to participate. you complete all the required sessions and take the knowledge quiz to earn your certificate. you will be entered to win a $ 50 gift card for $ 50. you can be entered with a gift card to earn a $ 500 gift card."

In [ ]:
import imaplib
import email
from transformers import BartForConditionalGeneration, BartTokenizer

model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

In [ ]:
mail = imaplib.IMAP4_SSL('smtp.gmail.com')
mail.login('gmail', 'password')
mail.select('inbox')


('OK', [b'341'])

In [ ]:
from datetime import datetime
today_date = datetime.now().strftime("%d-%b-%Y")

status, email_ids = mail.search(None, 'SINCE', today_date)
email_ids = email_ids[0].split()

def generate_summary(email_text):
    inputs = tokenizer(email_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

for email_id in email_ids[-10:]:
    status, msg_data = mail.fetch(email_id, "(RFC822)")
    raw_email = msg_data[0][1]
    email_message = email.message_from_bytes(raw_email)

    email_subject = email_message["Subject"]
    email_body = ""

    if email_message.is_multipart():
        for part in email_message.walk():
            if part.get_content_type() == "text/plain":
                email_body = part.get_payload(decode=True).decode()

    if email_body:
        summary = generate_summary(email_body)
        print(f"From: {sender}")
        print(f"Email Subject: {subject}")
        print(f"Generated Summary: {summary}")
        print(f"Sentiment Label: {email_label}")
        print(f"Sentiment Score: {score}")
        print("-" * 50)

mail.logout()

Email Subject: =?UTF-8?Q?=E2=80=9Cdata_scientist=E2=80=9D:_Deuglo_?=
 =?UTF-8?Q?Infosystem_Private_Limited?=
 =?UTF-8?Q?_-_Urgently_need=5F_Data_Scientist=5FBangalore_and_more?=
Generated Summary:
Your job alert for data scientist in Bengaluru. 6 new jobs match your preferences. Urgently need_ Data Scientist_Bangalore. Deuglo Infosystem Private Limited is looking for a Data Scientist. For more information, visit: http://www.linkedin.com/jobs/view/3744276550.
--------------------------------------------------
Email Subject: =?UTF-8?Q?=E2=80=9Cdata_engineer=E2=80=9D:_Antal_-_Data_Engineer_and_more?=
Generated Summary:
Your job alert for data engineer in Bengaluru. 19 new jobs match your preferences. Click here to see the full list of jobs available in the Bengaluru area. For confidential support call the Samaritans on 08457 90 90 90 or visit a local Samaritans branch, see www.samaritans.org for details.
--------------------------------------------------
Email Subject: Data Engineer, gt.s

('BYE', [b'LOGOUT Requested'])